In [1]:
from ultralytics import YOLO

# Load model
player_model = YOLO("models/player_tracking.pt")
player_model = player_model.to("cuda:1")

In [3]:
player_model_onnx = YOLO("models/player_tracking.onnx", task="detect")
frame = 'images/frame_1.jpg'

# Inference
results = player_model_onnx(frame, device="cuda:1")
# print(player_model_onnx)

Loading models/player_tracking.onnx for ONNX Runtime inference...


AttributeError: module 'onnxruntime' has no attribute 'InferenceSession'

In [17]:
import time 
import torch
import cv2

loader = cv2.VideoCapture("../videos/video_demo.mp4")

frame_count = 0
start_time = time.time()

while True:
    ret, frame = loader.read()
    if not ret:
        print("Exiting loop")
        break  # Exit the loop if there are no more frames
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Resize the frame to 640x640 to be divisible by 32
    frame_resized = cv2.resize(frame_rgb, (1280, 1280))
    # Convert the resized frame to a PyTorch tensor and normalize it (if required)
    frame_tensor = torch.from_numpy(frame_resized).float() / 255.0  # Normalize to [0, 1] range
    # Permute dimensions from (H, W, C) to (C, H, W) for PyTorch
    frame_tensor = frame_tensor.permute(2, 0, 1)
    # Add a batch dimension (1, C, H, W)
    frame_tensor = frame_tensor.unsqueeze(0)
    # Move the tensor to the correct device
    frame_tensor = frame_tensor.to("cuda:1")

    player_results = player_model_onnx(frame_tensor, imgsz=1280)
    
    frame_count += 1

end_time = time.time()
processing_time = end_time - start_time
fps = frame_count / processing_time
print(f"Processed {frame_count} frames in {processing_time:.2f} seconds. FPS: {fps:.2f}")

Loading models/player_tracking.onnx for ONNX Runtime inference...


/projects/demos/football_demo/football-venv/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:65: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


0: 1280x1280 24 players, 2002.5ms
Speed: 16.6ms preprocess, 2002.5ms inference, 3.3ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 24 players, 1943.2ms
Speed: 2.5ms preprocess, 1943.2ms inference, 3.1ms postprocess per image at shape (1, 3, 1280, 1280)



KeyboardInterrupt: 